In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://security.ubuntu.c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8EWA1OFT84NX|          5|            0|          0|   N|                Y|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|                N|
| RL5ESX231LZ0B|          5|            0|          0|   N|                Y|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|                Y|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|                Y|
|R13EPSFP5DODN5|          4|            0|          0|   N|                Y|
| R6RBP4HTE67SY|          5|            0|          0|   N|                Y|
|R15B3EU40RSU2W|          5|            0|          0|   N|                Y|
| RP4DD53A4ZJA2|          5|            0|          0|   N|                Y|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|     

In [7]:
# Filter the data and create a new DataFrame or table to retrieve all the rows 
# where the total_votes count is equal to or greater than 20 to 
# pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
total_votes_filtered_vine_df = vine_df.filter(df.total_votes >= 20)
total_votes_filtered_vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
|R3UUR313K5VVTL|          1|            1|         24|   N|                Y|
| RODE8K12S7148|          5|           41|         45|   N|     

In [15]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or 
# table to retrieve all the rows where the number of helpful_votes divided by total_votes 
# is equal to or greater than 50%.

helpful_total_ratio = total_votes_filtered_vine_df["helpful_votes"]/total_votes_filtered_vine_df["total_votes"]
helpful_votes_filtered_vine_df = total_votes_filtered_vine_df.filter(helpful_total_ratio >= .5)
helpful_votes_filtered_vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
| RODE8K12S7148|          5|           41|         45|   N|                Y|
|R2QZFYFUKP22SP|          5|           33|         37|   N|     

In [18]:
# create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)

paid_filtered_vines = helpful_votes_filtered_vine_df.filter(helpful_votes_filtered_vine_df["vine"] == "Y")
paid_filtered_vines.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3N0XV9267NOXV|          5|           41|         47|   Y|                N|
| RSA6JQ346JZHZ|          5|           55|         64|   Y|                Y|
|R1FXF4HRMCLG4C|          5|           69|         84|   Y|                N|
| RCTBWC3II42MG|          4|          113|        117|   Y|                N|
| RTMQM2CQ1XIZ0|          5|           21|         25|   Y|                N|
|R2CBJLCKQ612KU|          4|           27|         35|   Y|                N|
|R322QDGO4AV2B5|          5|           55|         65|   Y|                N|
| R2S7CXRDGQ6EE|          4|           36|         39|   Y|                N|
|R3ECMXK0SGR1VV|          5|           59|         64|   Y|                N|
|R3D55CPJ6J6Z78|          4|           15|         20|   Y|     

In [21]:
# retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_filtered_vines = helpful_votes_filtered_vine_df.filter(helpful_votes_filtered_vine_df["vine"] == "N")
unpaid_filtered_vines.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROI00QN8IS49I|          5|           83|         91|   N|                N|
|R13C5INE1RTZP6|          2|           26|         26|   N|                Y|
| RXVMMXCL67MZN|          4|          378|        383|   N|                Y|
|R33JJQWAUYBKD3|          5|          270|        280|   N|                Y|
|R31HQD6YXSQV1W|          3|           40|         48|   N|                Y|
|R33LQSF958O6K8|          5|           39|         41|   N|                Y|
|R3OIDSQJ84W7J1|          5|           21|         24|   N|                N|
| RODE8K12S7148|          5|           41|         45|   N|                Y|
|R2QZFYFUKP22SP|          5|           33|         37|   N|                Y|
| RFON10GAZKDL3|          3|           16|         20|   N|     

In [20]:
# Determine the total number of reviews for paid vines

paid_filtered_vines.count()


463

In [22]:
# Determine the total number of reviews for unpaid vines
unpaid_filtered_vines.count()


25094

In [25]:
# Determine the number of 5 star reviews for paid vines

five_star_paid_vines = paid_filtered_vines.filter(paid_filtered_vines["star_rating"] == 5)
five_star_paid_vines.count()

202

In [27]:
# Determine the number of 5 star reviews for unpaid vines

five_star_unpaid_vines = unpaid_filtered_vines.filter(unpaid_filtered_vines["star_rating"] == 5)
five_star_unpaid_vines.count()

12033

In [28]:
# Determine the % of 5-star reviews for paid vines
pct_five_star_paid = five_star_paid_vines.count() / paid_filtered_vines.count()
print(pct_five_star_paid)

0.43628509719222464


In [29]:
# Determine the % of 5-star reviews for unpaid vines
pct_five_star_unpaid = five_star_unpaid_vines.count() / unpaid_filtered_vines.count()
print(pct_five_star_unpaid)

0.4795170160197657


In [38]:
# Create df to show results
import pandas as pd
results_df = pd.DataFrame({
    " ": ["Total reviews", "5-star reviews", "5-star pct"],
    "Paid": [paid_filtered_vines.count(),five_star_paid_vines.count(), pct_five_star_paid ],
    "Unpaid": [unpaid_filtered_vines.count(), five_star_unpaid_vines.count(),pct_five_star_unpaid ]
})

results_df.set_index([" "], inplace = True)
display(results_df)

,Paid,Unpaid
,,
Total reviews,463.000000,25094.000000
5-star reviews,202.000000,12033.000000
5-star pct,0.436285,0.479517
